# DLProfile Example using l2hmc-qcd

## Set imports and neccessary environment variables

In [1]:
import pathlib
import os
import sys
import matplotlib.pyplot as plt
import warnings
import pprint
import pandas
import ntpath

In [2]:
VANIDL_DIR="{}".format(pathlib.Path(os.getcwd()).parent.absolute())
sys.path.insert(0, VANIDL_DIR)
warnings.filterwarnings('ignore')

In [3]:
os.environ["DARSHAN_DIR"] = "/soft/perftools/darshan/darshan-3.1.8"
os.environ["VANIDL_DIR"] = VANIDL_DIR

#### Formatting

In [4]:
pp = pprint.PrettyPrinter(indent=1)
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

## Create instrance of DL Profile and load the darshan file

In [5]:
from src.vanidl import VaniDL
profile = VaniDL()

In [6]:
#import shutil
#shutil.rmtree('/tmp/temp_analysis')

In [7]:

DATAPATH_INCLUDES = []
status = profile.Load("/home/dhari/darshan-logs/apps/l2hmc-qcd/l2hmc_run2_p8.darshan", data_paths_include=DATAPATH_INCLUDES)
if status:
    print("Darshan Trace loaded Successfully!")
else:
    print("Darshan Trace load Failed!")
    print(profile._error_str())

Loaded Pre-processed DXT DF from file: /tmp/temp_analysis/l2hmc_run2_p8_io_dxt_df.csv
Loaded Pre-processed DF from file: /tmp/temp_analysis/l2hmc_run2_p8_io_df.csv
Loaded Pre-processed Pattern file: /tmp/temp_analysis/l2hmc_run2_p8_pattern.json
Darshan Trace loaded Successfully!


## Use Profile object to analyze the darshan I/O trace.

### Verify if object works

The GetDXTAsDF() function enables users to perform analysis 

In [8]:
df = profile.GetDXTAsDF()
pp.pprint("Files used in the application")
pp.pprint(df['Filename'].unique().tolist())

'Files used in the application'
[]


In [9]:
df_normal = profile.GetTraceAsDF()
pp.pprint("Files used in the application")
pp.pprint(df_normal['Filename'].unique().tolist())

'Files used in the application'
['/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp025_f32_1909_1/checkpoints/checkpoint.tmp1aba5788d2bb4894a770124815ccea57',
 '/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp025_f32_1909_1/checkpoints/graph.pbtxt.tmpa14255dbe3834f45b758ec4703bc9b6d',
 '/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp025_f32_1909_1/checkpoints/model.ckpt-45_temp_6770bba4db834b5a9ef640a844920dde/part-00000-of-00001.data-00000-of-00001.tempstate6307127464834357184',
 '/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp025_f32_1909_1/checkpoints/model.ckpt-75.index',
 '/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp0

### Collect the summary of the Application

In [11]:
summary = profile.GetSummary()
print("\n")
print(color.BOLD + "Data Access Summary (from Darshan):"+ color.END)
print("Total Job time\t\t\t:\t{:0.2f} seconds".format(summary['job_time']))
#FIXME: calculate time per rank and then take max across it.
print("Time spent in I/O\t\t:\t{:0.2f} seconds".format(summary['total_io_time']))
print("% Time spent in I/O\t\t:\t{:0.2f}%".format(float(summary['total_io_time'])*100/summary['job_time']))
print("Total Data Accessed\t\t:\t{:0.2f} GB".format(float(summary['total_io_bytes'])/1024.0/1024.0/1024.0))
print("Data Access Modules used\t:\t{}".format(summary['io_interface_used']))
print("Data Operations\t\t\t:\t{}".format(summary['io_operations_used']))
print("# of files used\t\t\t:\t{}".format(len(summary['files_used'])))
print("# of MPI Ranks\t\t\t:\t{:0.0f} ranks".format(8))
      
     
print(color.UNDERLINE + "Access Pattern:"+ color.END)
print("\tSequential\t\t:\t{:0.2f}%".format(float(summary['access_pattern']['sequential'])))
print("\tConsecutive\t\t:\t{:0.2f}%".format(float(summary['access_pattern']['consecutive'])))
#An I/O op issued at an offset greater than where the previous I/O op ended.
#An I/O op issued at the offset immediately after the end of the previous I/O

print("\n")
print(color.BOLD + "Files Summary:"+ color.END)
print("File Types\t\t\t:\t{}".format(summary['file_used_summary']['types']))
print(color.UNDERLINE + "Dataset Size:"+ color.END)
print("\tTotal\t\t\t:\t{:0.3f} GB".format(float(summary['file_used_summary']['size']['total'])/1024.0/1024.0/1024.0))
print("\tMin,Max\t\t\t:\t{:0.3f} GB and {:0.3f} GB".format(float(summary['file_used_summary']['size']['min'])/1024.0/1024.0/1024.0,float(summary['file_used_summary']['size']['max'])/1024.0/1024.0/1024.0))
print("\tAverage\t\t\t:\t{:0.3f} GB".format(float(summary['file_used_summary']['size']['mean'])/1024.0/1024.0/1024.0))



Data Access Summary (from Darshan):
Total Job time			:	385.00 seconds
Time spent in I/O		:	4.14 seconds
% Time spent in I/O		:	1.08%
Total Data Accessed		:	0.34 GB
Data Access Modules used	:	['STDIO']
Data Operations			:	['write']
# of files used			:	43
# of MPI Ranks			:	8 ranks
Access Pattern:
	Sequential		:	nan%
	Consecutive		:	nan%


Files Summary:
File Types			:	[]
Dataset Size:
	Total			:	0.000 GB
	Min,Max			:	0.000 GB and 0.000 GB
	Average			:	0.000 GB


In [12]:
pp.pprint("Job time : {} seconds".format(profile.GetJobTime()))

'Job time : 385.0 seconds'


In [13]:
pp.pprint("Time spent by application on I/O: {} seconds".format(profile.GetIOTime()))

'Time spent by application on I/O: 4.140188 seconds'


### I/O time spent on each file

In [25]:
for file in df_normal['Filename'].unique():
    filename = os.path.splitext(ntpath.basename(file))[0]
    print("I/O time for file {}: {:0.2f} seconds".format(filename,profile.GetIOTime(filepath=file)))

I/O time for file checkpoint: 0.02 seconds
I/O time for file graph.pbtxt: 0.15 seconds
I/O time for file part-00000-of-00001.data-00000-of-00001: 0.28 seconds
I/O time for file model.ckpt-75: 0.02 seconds
I/O time for file checkpoint: 0.03 seconds
I/O time for file checkpoint: 0.03 seconds
I/O time for file model.ckpt-0: 0.02 seconds
I/O time for file part-00000-of-00001.data-00000-of-00001: 0.29 seconds
I/O time for file model.ckpt-90: 0.02 seconds
I/O time for file model.ckpt-45: 0.02 seconds
I/O time for file part-00000-of-00001.index: 0.02 seconds
I/O time for file part-00000-of-00001.index: 0.02 seconds
I/O time for file model.ckpt-90.meta: 0.16 seconds
I/O time for file checkpoint: 0.02 seconds
I/O time for file checkpoint: 0.02 seconds
I/O time for file part-00000-of-00001.index: 0.02 seconds
I/O time for file model.ckpt-45.meta: 0.12 seconds
I/O time for file part-00000-of-00001.data-00000-of-00001: 0.26 seconds
I/O time for file part-00000-of-00001.data-00000-of-00001: 0.32 se

### I/O Time spent per rank

In [15]:
for rank in df['Rank'].unique():
    print("I/O time for rank {}: {:0.2f} seconds".format(rank,profile.GetIOTime(rank=rank)))

In [16]:
"Total I/O performed by application: {:0.2f} GB".format(float(profile.GetIOSize())/1024.0/1024.0/1024.0)

'Total I/O performed by application: 0.34 GB'

### I/O performed on each file

In [31]:
for file in df_normal['Filename'].unique():
    filename = os.path.splitext(ntpath.basename(file))[0]
    print("I/O performed on file {}: {:0.2f} KB".format(filename,float(profile.GetIOSize(filepath=file))/1024.0))

I/O performed on file checkpoint: 1.12 KB
I/O performed on file graph.pbtxt: 7407.53 KB
I/O performed on file part-00000-of-00001.data-00000-of-00001: 35436.03 KB
I/O performed on file model.ckpt-75: 4.72 KB
I/O performed on file checkpoint: 0.93 KB
I/O performed on file checkpoint: 1.12 KB
I/O performed on file model.ckpt-0: 4.72 KB
I/O performed on file part-00000-of-00001.data-00000-of-00001: 35436.03 KB
I/O performed on file model.ckpt-90: 4.72 KB
I/O performed on file model.ckpt-45: 4.72 KB
I/O performed on file part-00000-of-00001.index: 4.72 KB
I/O performed on file part-00000-of-00001.index: 4.72 KB
I/O performed on file model.ckpt-90.meta: 5321.34 KB
I/O performed on file checkpoint: 0.74 KB
I/O performed on file checkpoint: 0.37 KB
I/O performed on file part-00000-of-00001.index: 4.72 KB
I/O performed on file model.ckpt-45.meta: 5321.34 KB
I/O performed on file part-00000-of-00001.data-00000-of-00001: 35436.03 KB
I/O performed on file part-00000-of-00001.data-00000-of-00001: 

In [33]:
for col in df_normal.columns:
    print(col)

Module
Rank
POSIX_OPENS
Filename
Mount
FS
POSIX_FILENOS
POSIX_DUPS
POSIX_READS
POSIX_WRITES
POSIX_SEEKS
POSIX_STATS
POSIX_MMAPS
POSIX_FSYNCS
POSIX_FDSYNCS
POSIX_RENAME_SOURCES
POSIX_RENAME_TARGETS
POSIX_RENAMED_FROM
POSIX_MODE
POSIX_BYTES_READ
POSIX_BYTES_WRITTEN
POSIX_MAX_BYTE_READ
POSIX_MAX_BYTE_WRITTEN
POSIX_CONSEC_READS
POSIX_CONSEC_WRITES
POSIX_SEQ_READS
POSIX_SEQ_WRITES
POSIX_RW_SWITCHES
POSIX_MEM_NOT_ALIGNED
POSIX_MEM_ALIGNMENT
POSIX_FILE_NOT_ALIGNED
POSIX_FILE_ALIGNMENT
POSIX_MAX_READ_TIME_SIZE
POSIX_MAX_WRITE_TIME_SIZE
POSIX_SIZE_READ_0_100
POSIX_SIZE_READ_100_1K
POSIX_SIZE_READ_1K_10K
POSIX_SIZE_READ_10K_100K
POSIX_SIZE_READ_100K_1M
POSIX_SIZE_READ_1M_4M
POSIX_SIZE_READ_4M_10M
POSIX_SIZE_READ_10M_100M
POSIX_SIZE_READ_100M_1G
POSIX_SIZE_READ_1G_PLUS
POSIX_SIZE_WRITE_0_100
POSIX_SIZE_WRITE_100_1K
POSIX_SIZE_WRITE_1K_10K
POSIX_SIZE_WRITE_10K_100K
POSIX_SIZE_WRITE_100K_1M
POSIX_SIZE_WRITE_1M_4M
POSIX_SIZE_WRITE_4M_10M
POSIX_SIZE_WRITE_10M_100M
POSIX_SIZE_WRITE_100M_1G
POSIX_SIZE_

In [38]:
for index, row in df_normal.iterrows():
    if row['Module'] == "STDIO":
        if row['STDIO_BYTES_READ'] != 0:
            print(row['Filename'], row['STDIO_F_READ_START_TIMESTAMP'],row['STDIO_F_READ_END_TIMESTAMP'])
        if row['STDIO_BYTES_WRITTEN'] != 0:
            print(row['Filename'], row['STDIO_F_WRITE_START_TIMESTAMP'],row['STDIO_F_WRITE_END_TIMESTAMP'])

/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp025_f32_1909_1/checkpoints/checkpoint.tmp1aba5788d2bb4894a770124815ccea57 337.589208 337.58979700000003
/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp025_f32_1909_1/checkpoints/graph.pbtxt.tmpa14255dbe3834f45b758ec4703bc9b6d 176.370032 176.381415
/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp025_f32_1909_1/checkpoints/model.ckpt-45_temp_6770bba4db834b5a9ef640a844920dde/part-00000-of-00001.data-00000-of-00001.tempstate6307127464834357184 322.93347 323.010064
/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp025_f32_1909_1/checkpoints/model.ckpt-75.index 351.900977 351.901356
/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11

In [45]:
ranks_set=[0,1,2,3,4,5,6,7]
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))
    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)
    if count == 1:
        print("")
    print("\r[{}] {}% {} of {} {} ".format(bar, percents, count, total, status), end='')
    if count == total:
        print("")
    os.sys.stdout.flush()
timestamps = []
data = []
pb_total = df_normal.count()['Module'];
i = 1
for index, row in df_normal.iterrows():
    if i % 100 == 0 or i == pb_total:
        progress(i, pb_total, status='Creating Darshan Timeline')
    i += 1
    if row['Module'] == "STDIO":
        args_val = {
            "Module": row['Module'],
            "Rank": row['Rank'],
            "Filename": row['Filename'],
            "STDIO_OPENS": row['STDIO_OPENS'],
            "STDIO_FDOPENS": row['STDIO_FDOPENS'],
            "STDIO_READS": row['STDIO_READS'],
            "STDIO_WRITES": row['STDIO_WRITES'],
            "STDIO_SEEKS": row['STDIO_SEEKS'],
            "STDIO_FLUSHES": row['STDIO_FLUSHES'],
            "STDIO_BYTES_WRITTEN": row['STDIO_BYTES_WRITTEN'],
            "STDIO_BYTES_READ": row['STDIO_BYTES_READ'],
            "STDIO_MAX_BYTE_READ": row['STDIO_MAX_BYTE_READ'],
            "STDIO_MAX_BYTE_WRITTEN": row['STDIO_MAX_BYTE_WRITTEN'],
            "STDIO_FASTEST_RANK": row['STDIO_FASTEST_RANK'],
            "STDIO_FASTEST_RANK_BYTES": row['STDIO_FASTEST_RANK_BYTES'],
            "STDIO_SLOWEST_RANK": row['STDIO_SLOWEST_RANK'],
            "STDIO_SLOWEST_RANK_BYTES": row['STDIO_SLOWEST_RANK_BYTES'],
            "STDIO_F_META_TIME": row['STDIO_F_META_TIME'],
            "STDIO_F_WRITE_TIME": row['STDIO_F_WRITE_TIME'],
            "STDIO_F_READ_TIME": row['STDIO_F_READ_TIME'],
            "STDIO_F_OPEN_START_TIMESTAMP": row['STDIO_F_OPEN_START_TIMESTAMP'],
            "STDIO_F_CLOSE_START_TIMESTAMP": row['STDIO_F_CLOSE_START_TIMESTAMP'],
            "STDIO_F_WRITE_START_TIMESTAMP": row['STDIO_F_WRITE_START_TIMESTAMP'],
            "STDIO_F_READ_START_TIMESTAMP": row['STDIO_F_READ_START_TIMESTAMP'],
            "STDIO_F_OPEN_END_TIMESTAMP": row['STDIO_F_OPEN_END_TIMESTAMP'],
            "STDIO_F_CLOSE_END_TIMESTAMP": row['STDIO_F_CLOSE_END_TIMESTAMP'],
            "STDIO_F_WRITE_END_TIMESTAMP": row['STDIO_F_WRITE_END_TIMESTAMP'],
            "STDIO_F_READ_END_TIMESTAMP": row['STDIO_F_READ_END_TIMESTAMP'],
            "STDIO_F_FASTEST_RANK_TIME": row['STDIO_F_FASTEST_RANK_TIME'],
            "STDIO_F_SLOWEST_RANK_TIME": row['STDIO_F_SLOWEST_RANK_TIME'],
            "STDIO_F_VARIANCE_RANK_TIME": row['STDIO_F_VARIANCE_RANK_TIME'],
            "STDIO_F_VARIANCE_RANK_BYTES": row['STDIO_F_VARIANCE_RANK_BYTES']
        }
        ranks = []
        if row['Rank'] != -1:
            ranks.append(int(row['Rank']))
        else:
            ranks.extend(list(ranks_set))
        for rank in ranks:
            if row['STDIO_BYTES_READ'] != 0:
                event_start = {"name": "darshan", "cat": row['Module'], "ph": "B",
                               "ts": int(float(row['STDIO_F_READ_START_TIMESTAMP']) * 1e6),
                               "pid": rank, "tid": 1,
                               "args": args_val
                               }
                event_end = {"ph": "E", "ts": int(float(row['STDIO_F_READ_END_TIMESTAMP']) * 1e6),
                             "pid": rank, "tid": 1,
                             "args": args_val
                             }
                timestamps.append(int(float(row['STDIO_F_READ_START_TIMESTAMP']) * 1e6))
                data.append(event_start)
                timestamps.append(int(float(row['STDIO_F_READ_END_TIMESTAMP']) * 1e6))
                data.append(event_end)
            if row['STDIO_BYTES_WRITTEN'] != 0:
                event_start = {"name": "darshan", "cat": row['Module'], "ph": "B",
                               "ts": int(float(row['STDIO_F_WRITE_START_TIMESTAMP']) * 1e6),
                               "pid": rank, "tid": 1,
                               "args": args_val
                               }
                event_end = {"ph": "E", "ts": int(float(row['STDIO_F_WRITE_END_TIMESTAMP']) * 1e6),
                             "pid": rank, "tid": 1,
                             "args": args_val
                             }
                timestamps.append(int(float(row['STDIO_F_WRITE_START_TIMESTAMP']) * 1e6))
                data.append(event_start)
                timestamps.append(int(float(row['STDIO_F_WRITE_END_TIMESTAMP']) * 1e6))
                data.append(event_end)

[============================================================] 100.0% 43 of 43 Creating Darshan Timeline 


In [47]:
data

[{'name': 'darshan',
  'cat': 'STDIO',
  'ph': 'B',
  'ts': 337589208,
  'pid': 0,
  'tid': 1,
  'args': {'Module': 'STDIO',
   'Rank': 0,
   'Filename': '/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp025_f32_1909_1/checkpoints/checkpoint.tmp1aba5788d2bb4894a770124815ccea57',
   'STDIO_OPENS': 1.0,
   'STDIO_FDOPENS': 0.0,
   'STDIO_READS': 0.0,
   'STDIO_WRITES': 1.0,
   'STDIO_SEEKS': 0.0,
   'STDIO_FLUSHES': 0.0,
   'STDIO_BYTES_WRITTEN': 1146,
   'STDIO_BYTES_READ': 0,
   'STDIO_MAX_BYTE_READ': 0.0,
   'STDIO_MAX_BYTE_WRITTEN': 1145.0,
   'STDIO_FASTEST_RANK': 0.0,
   'STDIO_FASTEST_RANK_BYTES': 0.0,
   'STDIO_SLOWEST_RANK': 0.0,
   'STDIO_SLOWEST_RANK_BYTES': 0.0,
   'STDIO_F_META_TIME': 0.022563999999999997,
   'STDIO_F_WRITE_TIME': 0.000589,
   'STDIO_F_READ_TIME': 0.0,
   'STDIO_F_OPEN_START_TIMESTAMP': 337.569495,
   'STDIO_F_CLOSE_START_TIMESTAMP': 337.589873,
   'STDIO_F_WRITE_START_TIMESTAMP': 337.58

In [8]:
profile.CreateChromeTimeline(location="/home/dhari", filename="l2hmc_run2_p8_io.json")

[============================================================] 100.0% 42 of 42 Creating DXT Timeline 


{'traceEvents': [{'name': 'darshan',
   'cat': 'STDIO',
   'ph': 'B',
   'ts': 140144304,
   'pid': 0,
   'tid': 1,
   'args': {'Module': 'STDIO',
    'Rank': 0,
    'Filename': '/gpfs/mira-home/dhari/code/learnIO/benchmarks/l2hmc-qcd/gauge_logs/2020_06_11/DEBUG_L16_b32_lf2_GaugeNetwork_aw00_qw01_dp025_f32_1909_1/checkpoints/events.out.tfevents.1591902661.nid00008',
    'STDIO_OPENS': 1.0,
    'STDIO_FDOPENS': 0.0,
    'STDIO_READS': 0.0,
    'STDIO_WRITES': 42.0,
    'STDIO_SEEKS': 0.0,
    'STDIO_FLUSHES': 6.0,
    'STDIO_BYTES_WRITTEN': 19585691,
    'STDIO_BYTES_READ': 0,
    'STDIO_MAX_BYTE_READ': 0.0,
    'STDIO_MAX_BYTE_WRITTEN': 19585690.0,
    'STDIO_FASTEST_RANK': 0.0,
    'STDIO_FASTEST_RANK_BYTES': 0.0,
    'STDIO_SLOWEST_RANK': 0.0,
    'STDIO_SLOWEST_RANK_BYTES': 0.0,
    'STDIO_F_META_TIME': 0.042486,
    'STDIO_F_WRITE_TIME': 0.08765099999999999,
    'STDIO_F_READ_TIME': 0.0,
    'STDIO_F_OPEN_START_TIMESTAMP': 140.10161100000002,
    'STDIO_F_CLOSE_START_TIMESTAMP': 0.